In [12]:
import psycopg2
import sqlite3
import pandas as pd
import access

# Configurações do PostgreSQL
pg_conn = psycopg2.connect(
    host=access.HOST,
    database=access.DATABASE_PG,
    user=access.DATABASE_PG_USER,
    password=access.DATABASE_PG_PWD
)

# Conexão SQLite
conn = sqlite3.connect('C:/ProjetosPython/Gerar_scripts/database/config_scripts.db')
cursor = conn.cursor()

# Função para adicionar coluna
def column_add(columns, column_name, column_type):
    columns.append((column_name, column_type))

# Função para obter estrutura da tabela no PostgreSQL
def get_pg_table_structure(table_name):
    with pg_conn.cursor() as cursor:
        cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = %s", (table_name,))
        return cursor.fetchall()

# Função para obter estrutura da tabela no PostgreSQL com colunas adicionadas
def get_pg_table_structure_with_added_column(table_name):
    columns = get_pg_table_structure(table_name)
    column_add(columns, 'CREATED_AT', 'TIMESTAMP_NTZ(9)')
    column_add(columns, 'UPDATED_AT', 'TIMESTAMP_NTZ(9)')
    column_add(columns, 'ID_DATABASE', 'VARCHAR(50)')
    return columns

# Função para criar tabela no Snowflake
def create_sf_table(table_name, columns):
    create_query = f"create or replace sequence {table_name}_SEQUENCE start with 1 increment by 1; CREATE OR REPLACE TABLE {table_name} ({', '.join([f'{col[0]} {col[1]}' for col in columns])});"
    print(create_query)

# Função para migrar tabela
def migrate_table(pg_table_name, sf_table_name):
    pg_columns_with_added = get_pg_table_structure_with_added_column(pg_table_name)
    create_sf_table(sf_table_name, pg_columns_with_added)

try:
    # Leitura das tabelas do SQLite
    query_tabelas_full = "SELECT * FROM config_tabelas_full WHERE ATIVO = 'true';"
    df_tabelas_full = pd.read_sql_query(query_tabelas_full, conn)

    for _, tabelas_full in df_tabelas_full.iterrows():
        SCHEMA = tabelas_full['SCHEMA']
        TABELA = tabelas_full['TABELA']
        
        # Exemplo de migração de uma tabela com a nova coluna 'CREATED_AT'
        migrate_table(TABELA, f"RESSARCIMENTO.CURATED_DATA.{TABELA}")
        
except Exception as err:
    print(err)


create or replace sequence RESSARCIMENTO.CURATED_DATA.motivos_fat_SEQUENCE start with 1 increment by 1; CREATE OR REPLACE TABLE RESSARCIMENTO.CURATED_DATA.motivos_fat (id integer, ds_motivo text, cod_motivo character varying, terminativo boolean, classificacao character varying, CREATED_AT TIMESTAMP_NTZ(9), UPDATED_AT TIMESTAMP_NTZ(9), ID_DATABASE VARCHAR(50));
create or replace sequence RESSARCIMENTO.CURATED_DATA.pbm_vendas_pedidos_SEQUENCE start with 1 increment by 1; CREATE OR REPLACE TABLE RESSARCIMENTO.CURATED_DATA.pbm_vendas_pedidos (id integer, pbm_vendas_id integer, pedido_id integer, quantidade integer, dthr_atualizacao timestamp without time zone, CREATED_AT TIMESTAMP_NTZ(9), UPDATED_AT TIMESTAMP_NTZ(9), ID_DATABASE VARCHAR(50));
create or replace sequence RESSARCIMENTO.CURATED_DATA.pbm_vendas_repostas_SEQUENCE start with 1 increment by 1; CREATE OR REPLACE TABLE RESSARCIMENTO.CURATED_DATA.pbm_vendas_repostas (id integer, pbm_vendas_id integer, quantidade_reposta integer, pro